# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.15/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f481cced8b0>
├── 'a' --> tensor([[-1.1264, -0.8324,  0.1765],
│                   [-0.4915,  1.8014,  0.6185]])
└── 'x' --> <FastTreeValue 0x7f481cced7c0>
    └── 'c' --> tensor([[ 1.0274, -1.1936,  1.4719, -1.0575],
                        [ 0.0724, -0.4779,  0.1842, -0.9586],
                        [ 0.1243, -0.7194, -0.3133,  0.8046]])

In [4]:
t.a

tensor([[-1.1264, -0.8324,  0.1765],
        [-0.4915,  1.8014,  0.6185]])

In [5]:
%timeit t.a

67.1 ns ± 0.0674 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f481cced8b0>
├── 'a' --> tensor([[ 0.1291, -0.8850, -1.2406],
│                   [ 0.8861,  0.7607, -0.0806]])
└── 'x' --> <FastTreeValue 0x7f481cced7c0>
    └── 'c' --> tensor([[ 1.0274, -1.1936,  1.4719, -1.0575],
                        [ 0.0724, -0.4779,  0.1842, -0.9586],
                        [ 0.1243, -0.7194, -0.3133,  0.8046]])

In [7]:
%timeit t.a = new_value

66.2 ns ± 0.0456 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.1264, -0.8324,  0.1765],
               [-0.4915,  1.8014,  0.6185]]),
    x: Batch(
           c: tensor([[ 1.0274, -1.1936,  1.4719, -1.0575],
                      [ 0.0724, -0.4779,  0.1842, -0.9586],
                      [ 0.1243, -0.7194, -0.3133,  0.8046]]),
       ),
)

In [10]:
b.a

tensor([[-1.1264, -0.8324,  0.1765],
        [-0.4915,  1.8014,  0.6185]])

In [11]:
%timeit b.a

57.9 ns ± 0.0533 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.7143, -0.4581,  0.0680],
               [ 0.9332,  0.8146, -0.0183]]),
    x: Batch(
           c: tensor([[ 1.0274, -1.1936,  1.4719, -1.0575],
                      [ 0.0724, -0.4779,  0.1842, -0.9586],
                      [ 0.1243, -0.7194, -0.3133,  0.8046]]),
       ),
)

In [13]:
%timeit b.a = new_value

498 ns ± 0.0669 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

806 ns ± 0.137 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.8 µs ± 35.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

143 µs ± 299 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

145 µs ± 499 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f47746c0e50>
├── 'a' --> tensor([[[-1.1264, -0.8324,  0.1765],
│                    [-0.4915,  1.8014,  0.6185]],
│           
│                   [[-1.1264, -0.8324,  0.1765],
│                    [-0.4915,  1.8014,  0.6185]],
│           
│                   [[-1.1264, -0.8324,  0.1765],
│                    [-0.4915,  1.8014,  0.6185]],
│           
│                   [[-1.1264, -0.8324,  0.1765],
│                    [-0.4915,  1.8014,  0.6185]],
│           
│                   [[-1.1264, -0.8324,  0.1765],
│                    [-0.4915,  1.8014,  0.6185]],
│           
│                   [[-1.1264, -0.8324,  0.1765],
│                    [-0.4915,  1.8014,  0.6185]],
│           
│                   [[-1.1264, -0.8324,  0.1765],
│                    [-0.4915,  1.8014,  0.6185]],
│           
│                   [[-1.1264, -0.8324,  0.1765],
│                    [-0.4915,  1.8014,  0.6185]]])
└── 'x' --> <FastTreeValue 0x7f47746c06d0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.5 µs ± 84.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f47f81d3400>
├── 'a' --> tensor([[-1.1264, -0.8324,  0.1765],
│                   [-0.4915,  1.8014,  0.6185],
│                   [-1.1264, -0.8324,  0.1765],
│                   [-0.4915,  1.8014,  0.6185],
│                   [-1.1264, -0.8324,  0.1765],
│                   [-0.4915,  1.8014,  0.6185],
│                   [-1.1264, -0.8324,  0.1765],
│                   [-0.4915,  1.8014,  0.6185],
│                   [-1.1264, -0.8324,  0.1765],
│                   [-0.4915,  1.8014,  0.6185],
│                   [-1.1264, -0.8324,  0.1765],
│                   [-0.4915,  1.8014,  0.6185],
│                   [-1.1264, -0.8324,  0.1765],
│                   [-0.4915,  1.8014,  0.6185],
│                   [-1.1264, -0.8324,  0.1765],
│                   [-0.4915,  1.8014,  0.6185]])
└── 'x' --> <FastTreeValue 0x7f47f8989220>
    └── 'c' --> tensor([[ 1.0274, -1.1936,  1.4719, -1.0575],
                        [ 0.0724, -0.4779,  0.1842, -0.9586],
                 

In [23]:
%timeit t_cat(trees)

30.6 µs ± 77.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

55.4 µs ± 64.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-1.1264, -0.8324,  0.1765],
                [-0.4915,  1.8014,  0.6185]],
       
               [[-1.1264, -0.8324,  0.1765],
                [-0.4915,  1.8014,  0.6185]],
       
               [[-1.1264, -0.8324,  0.1765],
                [-0.4915,  1.8014,  0.6185]],
       
               [[-1.1264, -0.8324,  0.1765],
                [-0.4915,  1.8014,  0.6185]],
       
               [[-1.1264, -0.8324,  0.1765],
                [-0.4915,  1.8014,  0.6185]],
       
               [[-1.1264, -0.8324,  0.1765],
                [-0.4915,  1.8014,  0.6185]],
       
               [[-1.1264, -0.8324,  0.1765],
                [-0.4915,  1.8014,  0.6185]],
       
               [[-1.1264, -0.8324,  0.1765],
                [-0.4915,  1.8014,  0.6185]]]),
    x: Batch(
           c: tensor([[[ 1.0274, -1.1936,  1.4719, -1.0575],
                       [ 0.0724, -0.4779,  0.1842, -0.9586],
                       [ 0.1243, -0.7194, -0.3133,  0.8046]],
         

In [26]:
%timeit Batch.stack(batches)

77.8 µs ± 47.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-1.1264, -0.8324,  0.1765],
               [-0.4915,  1.8014,  0.6185],
               [-1.1264, -0.8324,  0.1765],
               [-0.4915,  1.8014,  0.6185],
               [-1.1264, -0.8324,  0.1765],
               [-0.4915,  1.8014,  0.6185],
               [-1.1264, -0.8324,  0.1765],
               [-0.4915,  1.8014,  0.6185],
               [-1.1264, -0.8324,  0.1765],
               [-0.4915,  1.8014,  0.6185],
               [-1.1264, -0.8324,  0.1765],
               [-0.4915,  1.8014,  0.6185],
               [-1.1264, -0.8324,  0.1765],
               [-0.4915,  1.8014,  0.6185],
               [-1.1264, -0.8324,  0.1765],
               [-0.4915,  1.8014,  0.6185]]),
    x: Batch(
           c: tensor([[ 1.0274, -1.1936,  1.4719, -1.0575],
                      [ 0.0724, -0.4779,  0.1842, -0.9586],
                      [ 0.1243, -0.7194, -0.3133,  0.8046],
                      [ 1.0274, -1.1936,  1.4719, -1.0575],
                      [ 0.0724, -

In [28]:
%timeit Batch.cat(batches)

144 µs ± 148 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

322 µs ± 3.31 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
